## Crawl Articles

In [1]:
import re
from newspaper import Article, ArticleException

#기사 링크를 입력으로 받아 제목, 내용 그리고 링크를 파싱하여 dictionary 형태로 반환하는 함수
#input - 기사 url
#output - 기사의 dictionary 형태
def crawl_newspaper(url):
    
    try:
        article = Article(url,
                          language='ko',
                          memorize_articles=False,
                          fetch_images=False,
                          skip_bad_cleaner=True)
        article.download()
        article.parse()

        article_data = {}
        article_data['title'] = article.title.strip()  # 기사 제목
        article_data['text'] = article.text.replace("\n", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
        article_data['url'] = url  # 기사 링크

        return article_data
    except ArticleException as AE:
        raise AE
    

In [2]:
import requests
from bs4 import BeautifulSoup

#네이버에서 검색어를 이용해 해당 기간 내의 뉴스 리스트를 반환하는 함수
#input - 검색어, 기사일자, 뉴스리스트 시작페이지
#output - 뉴스 리스트
def get_newspaper_list(query,  # 검색어
                       date=None,  # 기사 일자
                       start_page=1  # 뉴스 리스트 페이지
                      ):
    
    url = 'https://search.naver.com/search.naver?&where=news&query={query}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&start={start_page}'  # query와 기간을 조건으로 한 검색 url 
    if not date:
        date = datetime.now()  # 오늘 날짜 instance 생성
        date = date.strftime("%Y.%m.%d")  # 날자 instance를 문자열 형태로 변환 ex) 2019.06.30
    
    res = requests.get(url.format(query=query, 
                                  start_date=date, 
                                  end_date=date, 
                                  start_page=(start_page - 1) * 10 + 1))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.text, 'lxml')
    
    # 최대 기사 페이지 수를 확인
    pages = [tag.get_text() for tag in soup.select(".paging > a, .paging > strong")] # 이전페이지,1,2,3페이지
    max_page = 0
    for page in pages:
        try:
            page = int(page)
            max_page = max(page, max_page)
        except:
            continue
            
    if start_page > max_page:  # 더이상 페이지가 없을 경우 빈 리스트를 반환
        return []
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출
    link_list = [a['href'] for a in soup.select(".news > ul.type01 > li > dl > dt > a")]  
    return link_list

In [3]:
from datetime import datetime, timedelta
from time import time

#get_newspaper_list 함수를 이용하여 시작일자 ~ 끝일자까지의 뉴스리스트를 반환
#input - 검색어 키워드, 검색 시기, 검색 종기, 검색 최대 페이지 수
#output - 검색 시기~종기 기간의 검색된 뉴스리스트 반환

def crawl_newspaper_by_query(query,  # 검색어 키워드
                             start_date=None,
                             end_date=None,
                             max_page=10): #원래10 임시로 1
    
    
    article_list = []  # 기사 리스트
    crawled_url = set()  # 검색한 기사 중복 검색을 위한 변수
        
    if not start_date:
        start_date = datetime.now()  # 오늘 날짜 instance 생성
    else:
        start_date = datetime.strptime(start_date, "%Y.%m.%d")  # 2019.06.30과 같은 문자열을 python instance로 변경
        
    if not end_date:
        end_date = datetime.now()
    else:
        end_date = datetime.strptime(end_date, "%Y.%m.%d")
    
    while start_date < end_date:
        print("Date: ", end_date)
        for cur_page in range(1, max_page):
            print("Page: {}".format(cur_page))
            
            # end_date 날짜의 기사 링크 수집
            link_list = get_newspaper_list(query, end_date.strftime("%Y.%m.%d"), start_page=cur_page)

            if not link_list:  # 더이상 수집할 뉴스가 없을 경우
                break
            
            for link in link_list:
                # 중복 수집 방지
                if link in crawled_url:
                    continue
                else:
                    crawled_url.update([link])
                
                try:
                    article = crawl_newspaper(link)
                    article['date'] = start_date
                    article_list.append(article)
                except ArticleException as AE:
                    continue
        
        end_date = end_date - timedelta(days=1)  # 수집할 날짜를 하루 전으로 변경
    return article_list

In [7]:
#뉴스기사 크롤링 실행
#input - 검색어, 검색시기, 검색종기, 검색할 최대페이지수
#output - 검색된 뉴스리스트 articles에 저장
#예시data articles[0] 출력
start = time()
articles = crawl_newspaper_by_query("반도체", "2019.01.01", "2019.09.09", max_page=2)
keyWord = "반도체" #추후 추가크롤링작업을 위해 검색어저장
#keyWords = []
#keyWords.append("자동차")
print(time() - start)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])

Date:  2019-09-09 00:00:00
Page: 1
Date:  2019-09-08 00:00:00
Page: 1
Date:  2019-09-07 00:00:00
Page: 1
Date:  2019-09-06 00:00:00
Page: 1
Date:  2019-09-05 00:00:00
Page: 1
Date:  2019-09-04 00:00:00
Page: 1
Date:  2019-09-03 00:00:00
Page: 1
Date:  2019-09-02 00:00:00
Page: 1
Date:  2019-09-01 00:00:00
Page: 1
Date:  2019-08-31 00:00:00
Page: 1
Date:  2019-08-30 00:00:00
Page: 1
Date:  2019-08-29 00:00:00
Page: 1
Date:  2019-08-28 00:00:00
Page: 1
Date:  2019-08-27 00:00:00
Page: 1
Date:  2019-08-26 00:00:00
Page: 1
Date:  2019-08-25 00:00:00
Page: 1
Date:  2019-08-24 00:00:00
Page: 1
Date:  2019-08-23 00:00:00
Page: 1
Date:  2019-08-22 00:00:00
Page: 1
Date:  2019-08-21 00:00:00
Page: 1
Date:  2019-08-20 00:00:00
Page: 1
Date:  2019-08-19 00:00:00
Page: 1
Date:  2019-08-18 00:00:00
Page: 1
Date:  2019-08-17 00:00:00
Page: 1
Date:  2019-08-16 00:00:00
Page: 1
Date:  2019-08-15 00:00:00
Page: 1
Date:  2019-08-14 00:00:00
Page: 1
Date:  2019-08-13 00:00:00
Page: 1
Date:  2019-08-12 00

Date:  2019-01-17 00:00:00
Page: 1
Date:  2019-01-16 00:00:00
Page: 1
Date:  2019-01-15 00:00:00
Page: 1
Date:  2019-01-14 00:00:00
Page: 1
Date:  2019-01-13 00:00:00
Page: 1
Date:  2019-01-12 00:00:00
Page: 1
Date:  2019-01-11 00:00:00
Page: 1
Date:  2019-01-10 00:00:00
Page: 1
Date:  2019-01-09 00:00:00
Page: 1
Date:  2019-01-08 00:00:00
Page: 1
Date:  2019-01-07 00:00:00
Page: 1
Date:  2019-01-06 00:00:00
Page: 1
Date:  2019-01-05 00:00:00
Page: 1
Date:  2019-01-04 00:00:00
Page: 1
Date:  2019-01-03 00:00:00
Page: 1
Date:  2019-01-02 00:00:00
Page: 1
1458.7215514183044
Total articles: 2,455
==================== Article sample ====================
{'title': '창립 50주년 KEC "글로벌 전력반도체 100년 기업 도약"', 'text': "올해 창립 50주년을 맞은 전력반도체 기업 KEC가 100년 기업 도약을 다짐했다.    KEC(대표 황창섭)는 9일 구미코(GUMICO) 전시장에서 비전선포식을 개최하고 '미래형 오토·인더스트리 글로벌 전력반도체 기업'으로 성장하겠다는 비전을 제시했다. 이를 위한 'KEC 비전 2025'를 발표하고 노동조합과 회사 미래를 위한 노사 상생을 선언했다.    KEC는 구미국가산업단지 1호 입주 기업이다. 이날 행사에는 구미시, 한국노총 전국금속노동조합연맹을 비롯한 내빈과 KEC그룹 임직원 등 약 700여명이

## Text preprocessing

In [8]:
#형태소 분류작업 (정규화, 어근화작업도 수행)
#input - 형태소 분류할 문장
#output - 분류된 형태소
#예시data 출력
from konlpy.tag import Okt
from collections import Counter
# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)


#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
#stopwords = set(["종가", "가격", "영업", "종목", "등락", "포인트", "호가"])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
stopwords = set([])
stopwords.add(keyWord)  #검색어 자체가 키워드에 포함되지 않게한다


for i, article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        
    words = tagger.pos(article['text'], norm=True, stem=True) #정규화, 어근화도 진행
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    
    # 기사 내 서로 다른 단어 개수가 10개 이상인 기사만 사용    
    if len(words) > 10:
        new_words_list.append((words, article))

C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


## Build term-document matrix

In [9]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


print(len(words_list))
print(len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

2455
10000
Term document matrix shape: (2455, 10000)
  (0, 9904)	0.03095198937014999
  (0, 9851)	0.04354881237525273
  (0, 9843)	0.02962128769115888
  (0, 9840)	0.0258690006389916
  (0, 9676)	0.10518441456358922
  (0, 9675)	0.06958778176927478
  (0, 9646)	0.043089523268673026
  (0, 9633)	0.045727263449437526
  (0, 9621)	0.028100369883530475
  (0, 9609)	0.0720777776020723
  (0, 9496)	0.02071717287837867
  (0, 9246)	0.12544025752665874
  (0, 9111)	0.07700425926278173
  (0, 9103)	0.05530365126982242
  (0, 9028)	0.11428479905390039
  (0, 9008)	0.04197442027689147
  (0, 8256)	0.1649398961998306
  (0, 8251)	0.06022051561939442
  (0, 8239)	0.038026957930874385
  (0, 8202)	0.0350614715211964
  (0, 8118)	0.07197535451396411
  (0, 8105)	0.041350621578632535
  (0, 8000)	0.04154574330125319
  (0, 7932)	0.030779872663931816
  (0, 7854)	0.05819568132029553
  :	:
  (2344, 2409)	0.09186887583987384
  (2344, 2383)	0.03068720704832545
  (2344, 1996)	0.06973798150137434
  (2344, 1985)	0.22090022280986846

## Topic modeling

In [10]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 10  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다. 가중치
H = nmf.components_ #특성

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (2455, 10)
H shape: (10, 10000)


In [12]:
keywordVal = 0 #다음 키워드 선정을 위해 비교를 위한 값저장
top_topic_keywords = []
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:12]:
        print(vocab[index],  end=' ')
        top_topic_keywords.append((vocab[index], H[k][index]))
    print("\n", "="*20)
    
top_topic_keywords = sorted(top_topic_keywords, key=lambda x:x[1], reverse=True)
for x,y in top_topic_keywords:
    print(x)
    
#for k in range(K):
#    for index in H[k].argsort()[::-1][:1]:
#        if(H[k][index] > keywordVal):
#            keywordVal = H[k][index]
#            nextKeyword = vocab[index]
        
#    print(nextKeyword)

0th topic
가격 전자 SK 하이닉스 메모리 수요 하락 전망 낸드 실적 시장 재고 
1th topic
수출 감소 달러 품목 중국 증가 연속 개월 수입 흑자 부진 지난해 
2th topic
시스템 전자 기술 사업 분야 삼성 부회장 파운드리 비메 모리 개발 산업 
3th topic
종목 거래 상승 승률 거래량 마감 전일 일보 강세 업종 백만원 금은 
4th topic
일본 규제 수출 소재 한국 정부 조치 불화수소 기업 허가 품목 지스트 
5th topic
클러스터 수도권 SK 하이닉스 용인 조성 용인시 유치 단지 산업 경기도 구미 
6th topic
영업 이익 매출 실적 계약 대비 감소 동기 전자 지난해 순이익 공시 
7th topic
물가 지수 하락 상승 전월 개월 대비 가격 유가 포인트 연속 수입 
8th topic
화웨이 중국 미국 기업 무역 제재 트럼프 통신 스마트폰 업체 중단 애플 
9th topic
경제 산업 성장 투자 경기 제조업 전망 정부 올해 증가 둔화 한국 
수출
일본
종목
화웨이
영업
이익
물가
감소
클러스터
거래
지수
중국
미국
하락
규제
가격
시스템
전자
매출
전자
기술
수출
경제
수도권
실적
소재
SK
SK
한국
하이닉스
상승
달러
메모리
하이닉스
용인
상승
산업
수요
사업
전월
하락
성장
투자
전망
분야
삼성
조성
용인시
계약
낸드
유치
승률
거래량
실적
시장
부회장
마감
품목
정부
파운드리
전일
경기
단지
제조업
중국
재고
조치
비메
모리
증가
전망
개발
대비
연속
개월
개월
감소
일보
동기
강세
산업
불화수소
대비
산업
경기도
기업
허가
업종
구미
수입
정부
올해
전자
백만원
금은
품목
기업
지난해
흑자
부진
가격
무역
순이익
증가
공시
지난해
제재
지스트
둔화
유가
한국
트럼프
통신
스마트폰
포인트
연속
수입
업체
중단
애플


In [13]:
#new_topic_words = []

#for index in H[0].argsort()[::-1][:]:
#    print(vocab[index], end=' ')

In [47]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:10]:
        print(words_list[index][1]['title'])        
    print("\n", "="*20)
    
#print("***** 검색키워드와 관련성 TOP10 기사 *****")
#for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
#    for index in W[:, k].argsort()[::-1][:2]:
#        print(words_list[index][1]['title'], ' ', words_list[index][1]['url'])  
    

0th topic
車배터리 1개월, 반도체 2개월뒤 생산차질… 물량확보팀 해외 급파
트럼프발 무역공포 확산...유럽증시 붕락, 반도체주 직격탄
외국인, 日 반도체 수출규제에도 삼성전자 주식 매수
삼성전자도 일본의 수출규제 위험 줄이려면 반도체 생산감축 불가피
[日, 경제도발]다음은 웨이퍼 규제 유력…日입맛 따라 반도체 차질
“일 수입 대체품 발굴”…반도체·소재 업체 총력전
[실시간 분석] SFA반도체, +3.1% 상승…2Q 영업익 83억
반도체, 일본 없인 안 된다? 제3국 조달 · 국산화 주력
日 맥주도 안 마신다…수입맥주 시장 1위서 3위로 추락
'무역공포 확산'...미국증시 연일 추락, FAANG · 반도체 · 소비주 '고전'

1th topic
국산화의 힘?…코세스, 반도체 위기 고조에도 끄떡없는 이유
[fnRASSI]SFA반도체(036540) 현재 16.93% ↑
반도체용 특수가스에 불 실란으로 인명피해 없어
최재성, 日 반도체 수출 통제ㆍ도쿄올림픽 불참 가능성 또 거론
[한일 경제전쟁] ‘제2의 반도체’ 배터리, 日 2차제재 초긴장…업계 공동대응 나선다
SEMI, 반도체 진로체험으로 미래 반도체 인재 확보
반도체 쇼크에 상장사 영업익 ‘뚝’…“하반기도 부진할듯”
국산 반도체·디스플레이 장비 실적 곤두박질…업황 하락·투자 실종 여파
한경연 “日 수출규제, 일반기계>석유제품>반도체 순 타격”
점점 ‘아베’에게 등 돌리는 日 반도체 소재 기업

2th topic
삼성 세계 낸드 점유율 34.9%로 1위…`반도체 코리아` 굳건

[2019년 상반기 실적결산]상반기 코스닥 순이익 12.2%↓…"반도체 및 중국 부진 영향"
홍남기, "반도체 포함 핵심소재·부품 관련 예산 2조원 책정"
원안위 "서울반도체에서 방사선 피폭사고 발생…2명 이상증상"
​[일본 수출규제] 진대제 회장은···'삼성 반도체 신화' 주역
최기영 서울대 교수, 과기정통부 장관 내정...반도체·AI 전문가
[직업교육 혁신현장을 가다]<2>입학이 곧 반도체 분야 취업, 폴리텍대학 청주캠퍼스 '반

## Visualization

In [48]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)


W shape: (722, 10)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 722 samples in 0.001s...
[t-SNE] Computed neighbors for 722 samples in 0.030s...
[t-SNE] Computed conditional probabilities for sample 722 / 722
[t-SNE] Mean sigma: 0.036584
[t-SNE] KL divergence after 250 iterations with early exaggeration: 62.328362
[t-SNE] KL divergence after 1000 iterations: 0.547080


In [49]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [50]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})



p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [51]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from sklearn.preprocessing import normalize

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob


In [52]:
def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

In [53]:
doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)

In [56]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn


print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 10, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
show(prepared_data)

(10, 10000)
10000


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8894/    [Ctrl-C to exit]


127.0.0.1 - - [21/Aug/2019 15:05:57] "GET / HTTP/1.1" 200 -



stopping Server...


## Simple search engine

In [55]:
query = '화성시'
query_index = word2id[query]  # 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
#print('논제로_독', nonzero_doc);
ranks = query_vec.toarray().flatten().argsort()[::-1]
#print('랭크', ranks);



for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


티브로드 지역채널 http://ch1.tbroad.com/content/view?parent_no=24&content_no=61&p_no=86282
[사진]반도체 기업과 간담회 갖는 서철모 화성시장 http://www.newspim.com/news/view/20190805000893
반도체 소재 국산화 위한 ‘반도체 센터’ 건립되나…화성시, 국산화 공조 앞장 약속 http://www.hankookilbo.com/news/npath/201908051701078739?did=NA
[한일 경제전쟁] '반도체 메카' 화성시 日규제에 기업과 '맞손' http://yna.kr/AKR20190805114400061?did=1195m
"반도체 부품 업체 지원 절실" https://www.ytn.co.kr/_ln/0115_201908052126177719
"반도체 부품 업체 지원 절실" https://www.ytn.co.kr/_ln/0115_201908060020320948
'일 경제침략' 반도체 피해 최소화, 경기도 정치권 뭉쳤다 http://www.incheonilbo.com/news/articleView.html?idxno=971414
[fn마켓워치] ‘LCD·반도체 검사장비 업체’ 아주하이텍 매각 개시 http://www.fnnews.com/news/201908191025003059
평택에 반도체 소재·부품단지 짓는다 http://news.mk.co.kr/newsRead.php?year=2019&no=625765
반도체사업장 소재 도내 5개 단체장 손잡다 http://www.kgnews.co.kr/news/articleView.html?idxno=557849


## Word Cloud

In [37]:
import random
import os
from konlpy.tag import Hannanum
import pytagcloud
import webbrowser

r = lambda: random.randint(0,255) # 0~255 사이의 임의의 숫자를 반환
color = lambda: (r(), r(), r()) # 0~255 사이의 임의의 숫자 3개를 반환

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [38]:
def get_tags(multiplier=5):     
    return [{ 'color': color(), 'tag': n, 'size': c*multiplier }\
                for n, c in wcloud]

In [39]:
def draw_cloud(tags, filename, fontname='Korean', size=(800, 800)):
    pytagcloud.create_tag_image(tags, filename, fontname=fontname, size=size)
    webbrowser.open(filename)

In [40]:
for k in range(K):
    wcloud = []
    for index in H[k].argsort()[::-1][:20]:
        wcloud.append((vocab[index], int(round((H[k][index])*30))))
    tags = get_tags(5)
    print(tags)
    filename = f'wordcloud_topic{k}.png'
    draw_cloud(tags, filename)
        

[{'color': (8, 57, 94), 'tag': '일본', 'size': 180}, {'color': (55, 248, 132), 'tag': '한국', 'size': 95}, {'color': (225, 209, 166), 'tag': '수출', 'size': 85}, {'color': (20, 136, 244), 'tag': '정부', 'size': 75}, {'color': (136, 234, 181), 'tag': '조치', 'size': 65}, {'color': (143, 174, 69), 'tag': '규제', 'size': 65}, {'color': (144, 243, 170), 'tag': '우리', 'size': 45}, {'color': (170, 147, 222), 'tag': '국가', 'size': 40}, {'color': (83, 120, 41), 'tag': '경제', 'size': 35}, {'color': (54, 157, 110), 'tag': '기업', 'size': 35}, {'color': (14, 186, 65), 'tag': '대한', 'size': 35}, {'color': (242, 40, 50), 'tag': '제외', 'size': 35}, {'color': (206, 90, 129), 'tag': '백색', 'size': 30}, {'color': (1, 231, 142), 'tag': '문제', 'size': 30}, {'color': (91, 245, 131), 'tag': '허가', 'size': 30}, {'color': (54, 219, 78), 'tag': '대해', 'size': 30}, {'color': (164, 61, 111), 'tag': '품목', 'size': 30}, {'color': (149, 197, 21), 'tag': 'WTO', 'size': 30}, {'color': (223, 24, 130), 'tag': '아베', 'size': 25}, {'color': (16

[{'color': (232, 115, 91), 'tag': '부회장', 'size': 145}, {'color': (150, 178, 89), 'tag': '전자', 'size': 100}, {'color': (250, 46, 246), 'tag': '삼성', 'size': 85}, {'color': (107, 141, 4), 'tag': '이재용', 'size': 55}, {'color': (42, 42, 138), 'tag': '사업', 'size': 50}, {'color': (142, 107, 54), 'tag': '사장', 'size': 35}, {'color': (192, 225, 196), 'tag': '일본', 'size': 30}, {'color': (130, 210, 116), 'tag': '출장', 'size': 30}, {'color': (48, 67, 229), 'tag': '방문', 'size': 30}, {'color': (128, 15, 108), 'tag': '장단', 'size': 30}, {'color': (213, 109, 69), 'tag': '회의', 'size': 30}, {'color': (112, 243, 151), 'tag': '재계', 'size': 25}, {'color': (49, 95, 99), 'tag': '회장', 'size': 20}, {'color': (10, 64, 186), 'tag': '부장', 'size': 20}, {'color': (46, 172, 185), 'tag': '위기', 'size': 20}, {'color': (27, 74, 30), 'tag': '비상', 'size': 20}, {'color': (202, 211, 87), 'tag': '캠퍼스', 'size': 20}, {'color': (5, 4, 243), 'tag': '파운드리', 'size': 20}, {'color': (26, 139, 20), 'tag': '부문', 'size': 20}, {'color': (17

## Re-Crawling

In [14]:


keyWords =[] #키워드로 포함하지 않을 단어에 포함 시키기위해 이전 키워드 저장
for k in range(K):
    #print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:1]:
        keyWords.append(vocab[index])
        #print(keyWords)
        
#for z in keyWords:
    #print(z)

    
keyWords.append(nextKeyword) #검색어는 키워드에서 제외

nextKeyword = keyWord + ' ' + nextKeyword
print('다음검색어 : ' , nextKeyword)
start = time()
articles = crawl_newspaper_by_query(nextKeyword, "2019.07.17", "2019.08.07", max_page=5)
print(time() - start)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])
keyWord = nextKeyword


다음검색어 :  반도체 일본
Date:  2019-08-07 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-06 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-05 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-04 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-03 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-02 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-01 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-31 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-30 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-29 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-28 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-27 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-26 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-25 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-24 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-23 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-22 00:00:00
Page: 1
Page:

In [15]:
#형태소 분류작업 (정규화, 어근화작업도 수행)
#input - 형태소 분류할 문장
#output - 분류된 형태소
#예시data 출력



#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
#stopwords = set(["글자"])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
#전 키워드들은 금지단어에 포함
stopwords.update(keyWords)


for i, article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        
    words = tagger.pos(article['text'], norm=True, stem=True)
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    
    # 기사 내 서로 다른 단어 개수가 10개 이상인 기사만 사용    
    if len(words) > 10:
        new_words_list.append((words, article))

0
100
200
300
400
500
600
700


In [16]:
print(len(words_list))
print(len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

776
10000
Term document matrix shape: (776, 10000)
  (0, 9928)	0.053756925185029274
  (0, 9842)	0.0664454049782818
  (0, 9841)	0.06933430959261871
  (0, 9830)	0.04024523914549214
  (0, 9733)	0.03792833864089843
  (0, 9728)	0.032084116517013705
  (0, 9710)	0.027955915616148668
  (0, 9707)	0.04272382875395169
  (0, 9606)	0.04290804582811963
  (0, 9469)	0.016551224776655986
  (0, 9328)	0.0678697454614564
  (0, 9306)	0.04582268314178832
  (0, 9105)	0.03843593688739474
  (0, 9080)	0.04309503183560276
  (0, 8994)	0.13677076738839106
  (0, 8911)	0.04407462648685858
  (0, 8885)	0.06655964867660549
  (0, 8605)	0.04901484524607295
  (0, 8596)	0.07291146904698653
  (0, 8517)	0.034104945714053744
  (0, 8486)	0.09359686542225573
  (0, 8361)	0.043477660472924874
  (0, 8226)	0.03535423862064486
  (0, 8155)	0.06325271221118839
  (0, 8056)	0.06914141514391804
  :	:
  (751, 1474)	0.032071121739699754
  (751, 1436)	0.05932596724418489
  (751, 1324)	0.02110375425931821
  (751, 1263)	0.10694552416547527
  

In [17]:
K = 5  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (776, 5)
H shape: (5, 10000)


In [18]:
keywordVal = 0 #다음 키워드 선정을 위해 비교를 위한 값저장
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')        
    print("\n", "="*20)
    
for k in range(K):
    for index in H[k].argsort()[::-1][:1]:
        if(H[k][index] > keywordVal):
            keywordVal = H[k][index]
            nextKeyword = vocab[index]
        
    print(nextKeyword)
    
print("***** 검색키워드와 관련성 TOP10 기사 *****")
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    for index in W[:, k].argsort()[::-1][:2]:
        print(words_list[index][1]['title'], ' ', words_list[index][1]['url'])  

0th topic
한국 수출 정부 미국 조치 국가 무역 제외 허가 규제 WTO 산업 장관 백색 한일 품목 우리 경제 통제 화이트리스트 
1th topic
의회 보복 정부 규탄 조치 철회 촉구 경제 강제 결의 배상 징용 판결 의원 일본정부 피해자 즉각 우리 행위 의장 
2th topic
지원 기업 피해 대응 자금 대책 부품 수출 규제 방안 관련 중소기업 산업 마련 경기도 지역 센터 예산 긴급 협력 
3th topic
이익 하이닉스 메모리 SK 실적 낸드 생산 수요 하락 수출 업황 감산 투자 대비 하반기 매출 시장 삼성 증가 회복 
4th topic
업체 불화수소 생산 업계 국내 SK 국산 공정 장비 하이닉스 공급 규제 한국 대체 제품 중국 테스트 확보 부품 지스트 
한국
한국
한국
한국
한국
***** 검색키워드와 관련성 TOP10 기사 *****
미·중 사이 한국은 누구 친구냐, 일본이 경고사격 날린 것   http://news.joins.com/article/olink/23129638
[단독] 일본 '백색국가' 총구, 한국 미래 모빌리티 까지 정조준   https://www.hankyung.com/economy/article/201907246260G
국회 방미단, 미국서 ‘일본 수출규제’ 대응 의원외교   http://www.sisaweek.com/news/articleView.html?idxno=124529
산업장관 “‘한국 수출통제체제 일본보다 우수’ 평가”…日규제 조속 철회 촉구   http://news.kbs.co.kr/news/view.do?ncd=4255642&ref=A
[日, 경제도발]日"대만 반도체 쓰면 된다…韓대응 영향 없어"   http://news.mt.co.kr/mtview.php?no=2019080416150333938
日 반도체소재 수출 규제에도 外人, 삼성·하이닉스 2兆 매수   http://www.dt.co.kr/contents.html?article_no=2019080502101132048001&ref=naver
유명희, 美 삼

In [27]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn

#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob

def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)


print(vocab[0]) 

print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 10, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
#show(prepared_data)

AB
(5, 7695)
7695


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


KeyboardInterrupt: 

In [19]:
#3번째크롤링

keyWords =[] #키워드로 포함하지 않을 단어에 포함 시키기위해 이전 키워드 저장
for k in range(K):
    #print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:1]:
        keyWords.append(vocab[index])
        #print(keyWords)
        
#for z in keyWords:
    #print(z)
        

keyWords.append(nextKeyword) #추가된 검색키워드는 제외리스트 포함

nextKeyword = keyWord + ' ' + nextKeyword
keyWord = nextKeyword
print('다음검색어 : ' , nextKeyword)
start = time()
articles = crawl_newspaper_by_query(nextKeyword, "2019.07.27", "2019.08.07", max_page=5)
print(time() - start)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])


다음검색어 :  반도체 일본 한국
Date:  2019-08-07 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-06 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-05 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-04 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-03 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-02 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-08-01 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-31 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-30 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-29 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-28 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
232.71326899528503
Total articles: 407
==================== Article sample ====================
{'title': '일본, 오늘 ‘백색국가 한국 제외’ 공포…품목 규제 범위는?', 'text': "이슈 일본, 백색국가서 한국 제외 일본, 오늘 ‘백색국가 한국 제외’ 공포…품목 규제 범위는? 뉴스광장 1부 입력 2019.08.07 (06:09) 수정 2019.08.07 (07:18)  댓글 1  좋아요  스크랩하기  공유하기 글씨 크게보기  가  글씨 작게보기  고화질 표준화질 키보드 컨트롤 안내 동영상영역 시작 동

In [20]:
#형태소 분류작업 (정규화, 어근화작업도 수행)
#input - 형태소 분류할 문장
#output - 분류된 형태소
#예시data 출력

#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
#stopwords = set(["글자"])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
#전 키워드들은 금지단어에 포함
stopwords.update(keyWords)


for i, article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        
    words = tagger.pos(article['text'], norm=True, stem=True)
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    
    # 기사 내 서로 다른 단어 개수가 10개 이상인 기사만 사용    
    if len(words) > 10:
        new_words_list.append((words, article))

0
100
200
300
400


In [21]:
print(len(words_list))
print(len(vocab))
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

K = 5  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

keywordVal = 0 #다음 키워드 선정을 위해 비교를 위한 값저장
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')        
    print("\n", "="*20)
    
for k in range(K):
    for index in H[k].argsort()[::-1][:1]:
        if(H[k][index] > keywordVal):
            keywordVal = H[k][index]
            nextKeyword = vocab[index]
        
    print(nextKeyword)
    
print("***** 검색키워드와 관련성 TOP10 기사 *****")
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    for index in W[:, k].argsort()[::-1][:2]:
        print(words_list[index][1]['title'], ' ', words_list[index][1]['url'])  

407
8669
Term document matrix shape: (407, 8669)
  (0, 8589)	0.030676661666934217
  (0, 8569)	0.0855208205295929
  (0, 8527)	0.07807810195917997
  (0, 8512)	0.038069258801514745
  (0, 8392)	0.061353323333868434
  (0, 8348)	0.516802153912537
  (0, 8281)	0.038192785824845935
  (0, 8266)	0.055779623456357556
  (0, 8133)	0.08003252491953862
  (0, 8026)	0.21238593923246804
  (0, 8016)	0.03428183577843403
  (0, 7984)	0.07050929075205056
  (0, 7957)	0.19741442979654938
  (0, 7808)	0.062445687288027005
  (0, 7796)	0.04278828670306316
  (0, 7749)	0.02763271467666736
  (0, 7729)	0.04842135390615338
  (0, 7652)	0.0573739117499027
  (0, 7624)	0.06568382055328627
  (0, 7599)	0.019519525489794992
  (0, 7542)	0.03337810733823305
  (0, 7448)	0.02514566392315164
  (0, 7445)	0.0573739117499027
  (0, 7368)	0.03312101028672163
  (0, 7036)	0.03610623538873878
  :	:
  (398, 1246)	0.06847959027817481
  (398, 1187)	0.1545716263155695
  (398, 1072)	0.04409452620033177
  (398, 1030)	0.12043379824643248
  (398, 

In [ ]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob

def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)


print(vocab[0]) 

print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 10, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
#show(prepared_data)